In [10]:
#import libraries
import rasterio
import numpy as np
import pandas as pd
from rasterio.warp import calculate_default_transform, reproject, Resampling
import folium
import os
import matplotlib.pyplot as plt
import geopandas as gpd
print("Libraries imported!")

Libraries imported!


In [2]:
# Paths
s2_path = 'data/sentinel2_sr_amazon_2023.tif'
dem_path = 'data/dem.tif'
gfw_path = 'data/gfw_loss_2023.tif'
output_dir = 'data/processed'
os.makedirs(output_dir, exist_ok=True)

print("Paths defined!")

Paths defined!


In [3]:
# Load Sentinel-2
try:
    with rasterio.open(s2_path) as s2:
        b4 = s2.read(1).astype(float)  # Red
        b8 = s2.read(2).astype(float)  # Near-infrared
        transform = s2.transform
        crs = s2.crs
        profile = s2.profile
    print("Sentinel-2 SR loaded!")
except Exception as e:
    print(f"Failed to load Sentinel-2: {e}")
    raise

Sentinel-2 SR loaded!


In [4]:
# Calculate NDVI
ndvi = np.where((b8 + b4) != 0, (b8 - b4) / (b8 + b4), 0)
profile.update(count=1, dtype=rasterio.float32)

# Save NDVI
ndvi_path = os.path.join(output_dir, 'ndvi.tif')
with rasterio.open(ndvi_path, 'w', **profile) as dst:
    dst.write(ndvi, 1)
print(f"NDVI saved as {ndvi_path}")

/var/folders/5k/0xhc217n6n1_kbh5sblmhcpm0000gn/T/ipykernel_66315/4094520342.py:2: RuntimeWarning: invalid value encountered in divide
  ndvi = np.where((b8 + b4) != 0, (b8 - b4) / (b8 + b4), 0)


NDVI saved as data/processed/ndvi.tif


In [5]:
# Load and reproject DEM
try:
    with rasterio.open(dem_path) as dem:
        elevation = dem.read(1)
        dem_transform = dem.transform
        dem_crs = dem.crs
    print("DEM loaded!")
except Exception as e:
    print(f"Failed to load DEM: {e}")
    raise

elevation_reproj = np.zeros_like(ndvi)
transform, width, height = calculate_default_transform(dem_crs, crs, dem.width, dem.height, *dem.bounds)
reproject(
    source=elevation,
    destination=elevation_reproj,
    src_transform=dem_transform,
    src_crs=dem_crs,
    dst_transform=transform,
    dst_crs=crs,
    resampling=Resampling.bilinear
)
slope = np.gradient(elevation_reproj)[0]
print("Slope calculated!")

DEM loaded!
Slope calculated!


In [6]:
# Load and resample GFW
try:
    with rasterio.open(gfw_path) as gfw:
        loss = gfw.read(1)
        gfw_transform = gfw.transform
        gfw_crs = gfw.crs
    print("GFW loaded!")
except Exception as e:
    print(f"Failed to load GFW: {e}")
    raise

loss_reproj = np.zeros_like(ndvi, dtype=np.uint8)
reproject(
    source=loss,
    destination=loss_reproj,
    src_transform=gfw_transform,
    src_crs=gfw_crs,
    dst_transform=transform,
    dst_crs=crs,
    resampling=Resampling.nearest
)
labels = np.where(loss_reproj > 0, 1, 0)  # 1 = loss, 0 = no loss
print("Labels created!")

GFW loaded!
Labels created!


In [7]:
# Create ML dataset
points = 500  # For 0.5°×0.5°
h, w = ndvi.shape
x, y = np.random.randint(0, h, points), np.random.randint(0, w, points)
data = {
    'NDVI': [ndvi[i, j] for i, j in zip(x, y)],
    'Elevation': [elevation_reproj[i, j] for i, j in zip(x, y)],
    'Slope': [slope[i, j] for i, j in zip(x, y)],
    'Deforestation': [labels[i, j] for i, j in zip(x, y)]
}
df = pd.DataFrame(data)
csv_path = os.path.join(output_dir, 'ml_dataset.csv')
df.to_csv(csv_path)
print(f"Dataset saved as {csv_path}")

Dataset saved as data/processed/ml_dataset.csv


In [13]:
# Paths
ndvi_path = 'data/processed/ndvi.tif'
output_dir = 'data/processed'
png_path = os.path.join(output_dir, 'ndvi.png')
map_path = os.path.join(output_dir, 'ndvi_map.html')
# Load NDVI
try:
    with rasterio.open(ndvi_path) as src:
        ndvi = src.read(1)
    print("NDVI loaded for PNG conversion!")
except Exception as e:
    print(f"Failed to load NDVI: {e}")
    raise

# Plot NDVI as PNG
plt.figure(figsize=(10, 10))
plt.imshow(ndvi, cmap='RdYlGn', vmin=-1, vmax=1)  # NDVI range: -1 to 1
plt.colorbar(label='NDVI')
plt.axis('off')
plt.savefig(png_path, bbox_inches='tight', transparent=True, dpi=100)
plt.close()
print(f"NDVI saved as {png_path}")

NDVI loaded for PNG conversion!
NDVI saved as data/processed/ndvi.png


In [14]:
# Visualize NDVI
m = folium.Map(location=[-7.25, -56.25], zoom_start=10)
folium.GeoJson(
    data={
        'type': 'Feature',
        'geometry': {
            'type': 'Polygon',
            'coordinates': [[[-56.5, -7.5], [-56.5, -7], [-56, -7], [-56, -7.5], [-56.5, -7.5]]]
        }
    }
).add_to(m)
# Add NDVI PNG as ImageOverlay
folium.raster_layers.ImageOverlay(
    image=png_path,
    bounds=[[-7.5, -56.5], [-7, -56]],
    opacity=0.7,
    interactive=True
).add_to(m)

# Save map
m.save(map_path)
print(f"Map saved as {map_path}")

Map saved as data/processed/ndvi_map.html
